In [2]:
import cv2
import numpy as np
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing_styles = mp.solutions.drawing_styles
import pandas as pd


In [3]:
cap = cv2.VideoCapture(0)
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('footInfrontOfOther.avi', fourcc, 20.0, (640,  480))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive  (stream end?). Exiting ...")
        break
    out.write(frame)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break
# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
print("hello world")

In [3]:
def get_coordinates(landmarks):
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x ,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x , landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x , landmarks[mp_pose.PoseLandmark.LEFT_HIP].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x , landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x , landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x , landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y]
    right_shoulder =[landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x , landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x , landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y]
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x , landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y]
    right_wrist =[landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x , landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x , landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x , landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].y]
    left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x , landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y]
    right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x , landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y]
    return [
        [left_shoulder,left_elbow,left_wrist],
        [right_shoulder,right_elbow,right_wrist],
        [left_hip,left_shoulder,left_elbow],
        [right_hip,right_shoulder,right_elbow],
        [left_shoulder,left_hip,left_knee],
        [right_shoulder,right_elbow,right_knee],
        [left_hip,left_knee,left_ankle],
        [right_hip,right_knee,right_ankle],
        [left_knee,left_ankle,left_foot_index],
        [right_knee,right_ankle,right_foot_index],
    ]

In [4]:

landmarks = ['class']
for val in range(1,33+1):
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [5]:
def calculate_angles(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle

In [6]:
def get_all_angles(required_coordinates):
    results = []
    for coordinate in required_coordinates:
        results.append(calculate_angles(coordinate[0],coordinate[1],coordinate[2]))
    return results

In [ ]:
cap = cv2.VideoCapture('360.avi')
with mp_holistic.pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.


    k=cv2.waitKey(1)
    if k == 117:
      export_landmarks(results,'up')
    if k == 100:
      export_landmarks(results,'down')
    cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

In [ ]:
# For webcam input:
cap = cv2.VideoCapture("360.avi")
with mp_pose.Pose(
    min_detection_confidence=0.5,
    smooth_landmarks = True,
    # model_complexity = 2,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    results_temp = results.pose_landmarks
    # print(results_temp[0][1][10:])
    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results_temp,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

In [ ]:
pose_landmarks = mp.solutions.pose.PoseLandmark
pose_landmarks


### Lean Model

In [ ]:
import pickle
with open('lean.pkl','rb') as f:
    model1 = pickle.load(f)

In [ ]:
# For webcam input:
counter = 0
current_stage = ''
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # mp_drawing.draw_landmarks(
    #     image,
    #     results.face_landmarks,
    #     mp_holistic.FACEMESH_CONTOURS,
    #     landmark_drawing_spec=None,
    #     connection_drawing_spec=mp_drawing_styles
    #     .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    try:
        row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        X = pd.DataFrame([row],columns=landmarks[1:])
        body_lean_class = model.predict(X)[0]
        body_lean_prob = model.predict_proba(X)[0]
        print(body_lean_class,body_lean_prob)

        if body_lean_class == 'neutral' and body_lean_prob[body_lean_prob.argmax()] >= 0.7:
            current_stage = 'neutral'
            # print("huuuuuu")
        elif body_lean_class == 'left' and body_lean_prob[body_lean_prob.argmax()] >= 0.7:
            current_stage = 'left'
        elif body_lean_class == 'right' and body_lean_prob[body_lean_prob.argmax()] >= 0.7:
            current_stage = 'right'
        
        # Get status box
        cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1)

        #Display class
        cv2.putText(image,'CLASS'
            ,(95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,body_lean_class.split(' ')[0]
        , (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        #Display Probability
        cv2.putText(image,'PROB',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,str(round(body_lean_prob[np.argmax(body_lean_prob)],2))
        ,(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        # #Display Probability
        # cv2.putText(image,'COUNT',(180,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        # cv2.putText(image,str(counter),(175,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

    except Exception as e:
        pass

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Holistic',image ) #cv2.flip(image, 1)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

### Stance Model

In [ ]:
with open('stance.pkl','rb') as f:
    model2 = pickle.load(f)

In [ ]:
# For webcam input:
counter = 0
current_stage = ''
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    try:
        row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        X = pd.DataFrame([row],columns=landmarks[1:])
        body_lean_class = model2.predict(X)[0]
        body_lean_prob = model2.predict_proba(X)[0]
        print(body_lean_class,body_lean_prob)

        if body_lean_class == 'neutral' and body_lean_prob[body_lean_prob.argmax()] >= 0.7:
            current_stage = 'neutral'
            # print("huuuuuu")
        elif body_lean_class == 'left' and body_lean_prob[body_lean_prob.argmax()] >= 0.7:
            current_stage = 'left'
        elif body_lean_class == 'right' and body_lean_prob[body_lean_prob.argmax()] >= 0.7:
            current_stage = 'right'
        
        # Get status box
        cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1)

        #Display class
        cv2.putText(image,'CLASS'
            ,(95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,body_lean_class.split(' ')[0]
        , (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        #Display Probability
        cv2.putText(image,'PROB',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,str(round(body_lean_prob[np.argmax(body_lean_prob)],2))
        ,(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        # #Display Probability
        # cv2.putText(image,'COUNT',(180,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        # cv2.putText(image,str(counter),(175,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

    except Exception as e:
        pass

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Holistic',image ) #cv2.flip(image, 1)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

### Running the MediaPipe Model

In [ ]:
import pickle
with open('deadlift.pkl','rb') as f:
    model = pickle.load(f)

In [ ]:
landmarks = ['class']
for val in range(1,33+1):
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [ ]:
# For webcam input:
counter = 0
current_stage = ''
current_lean_status = ''
prev_lean_status = 'neutral'
current_stance_status = ''
prev_stance_status = 'normal'

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('final_demo.avi', fourcc, 20.0, (640,480))



cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # mp_drawing.draw_landmarks(
    #     image,
    #     results.face_landmarks,
    #     mp_holistic.FACEMESH_CONTOURS,
    #     landmark_drawing_spec=None,
    #     connection_drawing_spec=mp_drawing_styles
    #     .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    try:
        
      row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
      #position
      X1 = pd.DataFrame([row],columns=landmarks[1:])
      #lean
      X2 = pd.DataFrame([row],columns=landmarks[1:])
      #Stance
      X3 = pd.DataFrame([row],columns=landmarks[1:])

      body_language_class = model.predict(X1)[0]
      body_language_prob = model.predict_proba(X1)[0]

      lean_class = model1.predict(X2)[0]
      lean_prob = model1.predict_proba(X2)[0]
      # print(body_language_class,body_language_prob)

      stance_class = model2.predict(X3)[0]
      stance_prob = model2.predict_proba(X3)[0]
      ################################################
      if body_language_class == 'down' and body_language_prob[body_language_prob.argmax()] >= 0.7:
          current_stage = 'down'
          # print("huuuuuu")
      elif current_stage == 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] >= 0.7:
          current_stage = 'up'
          counter += 1
          print(counter)
          # print("this was hit!")

      ###################################################
      if lean_class == 'neutral' and lean_prob[lean_prob.argmax()] >= 0.7:

          current_lean_status = 'neutral'
      elif lean_class == 'left' and lean_prob[lean_prob.argmax()] >= 0.7:
          current_lean_status = 'left'
      elif lean_class == 'right' and lean_prob[lean_prob.argmax()] >= 0.7:
          current_lean_status = 'right'
      

      ######################################################
      if stance_class == 'normal' and stance_prob[stance_prob.argmax()] >= 0.7:
          current_stance_status = 'normal'
          # print("huuuuuu")
      elif stance_class == 'wide' and stance_prob[stance_prob.argmax()] >= 0.7:
          current_stance_status = 'wide'
      elif stance_class == 'narrow' and stance_prob[stance_prob.argmax()] >= 0.7:
          current_stance_status = 'narrow'  
      ######################################################
      #Extract Joint coordinates
      my_landmarks = results.pose_landmarks.landmark
      joint_coordinates = get_coordinates(my_landmarks)
      #calculate joint angles
      req_angles = get_all_angles(joint_coordinates)
      req_angles = np.around(req_angles,2)
      #display joint angles
      for i in range(len(req_angles)):
          # print(len(joint_coordinates))
          cv2.putText(
              image,
              str(req_angles[i]),
              tuple(np.multiply(joint_coordinates[i][1],[640, 480]).astype(int)),
              cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA
          )
      ###########################
      #Display for Position
      #Get status box
      cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1)

      #Display class
      cv2.putText(image,'CLASS'
          ,(95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
      cv2.putText(image,body_language_class.split(' ')[0]
      , (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

      #Display Probability
      cv2.putText(image,'PROB',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
      cv2.putText(image,str(round(body_language_prob[np.argmax(body_language_prob)],2))
      ,(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

      #Display Probability
      cv2.putText(image,'COUNT',(180,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
      cv2.putText(image,str(counter),(175,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
      ############################

      ############################
      # Display for Lean Status
      # Get status box
      cv2.rectangle(image,(250,0),(500,60),(245,117,16),-1)

      #Display class
      cv2.putText(image,'CLASS'
          ,(95+ 250 ,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
      cv2.putText(image,lean_class.split(' ')[0]
      , (90 + 250 ,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

      #Display Probability
      cv2.putText(image,'PROB',(15 + 250 ,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
      cv2.putText(image,str(round(lean_prob[np.argmax(lean_prob)],2))
      ,(10 + 250 ,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
      #############################

      #############################
      # Display for Stance Status
      # Get status box
      cv2.rectangle(image,(425,0),(640,60),(245,117,16),-1)

      #Display class
      cv2.putText(image,'CLASS'
          ,(95+ 425 ,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
      cv2.putText(image,stance_class.split(' ')[0]
      , (90 + 425 ,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

      #Display Probability
      cv2.putText(image,'PROB',(15 + 425 ,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
      cv2.putText(image,str(round(stance_prob[np.argmax(stance_prob)],2))
      ,(10 + 425 ,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
      #############################
    
    except Exception as e:
        print('error',e)
        pass

    # Flip the image horizontally for a selfie-view display.
    flipped_frame = cv2.flip(image, 0)

    out.write(image)
    cv2.imshow('MediaPipe Holistic',image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

In [ ]:
ff = [12.2342,5262.234234234]
np.around(ff,2)

In [ ]:
import cv2

# initialize video capture object
cap = cv2.VideoCapture(0)

# define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('turning_one.avi', fourcc, 20.0, (640, 480))

# loop over frames from the video stream
while cap.isOpened():
    # read the next frame from the video stream
    ret, frame = cap.read()

    if ret:
        # write the frame to the output file
        out.write(frame)

        # display the resulting frame
        cv2.imshow('frame', frame)

        # exit if the user presses 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# release resources
cap.release()
out.release()
cv2.destroyAllWindows()


In [8]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    try:
      ######################################################
      #Extract Joint coordinates
      my_landmarks = results.pose_landmarks.landmark
      joint_coordinates = get_coordinates(my_landmarks)
      #calculate joint angles
      req_angles = get_all_angles(joint_coordinates)
      req_angles = np.around(req_angles,2)
      #display joint angles
      for i in range(len(req_angles)):
          # print(len(joint_coordinates))
          cv2.putText(
              image,
              str(req_angles[i]),
              tuple(np.multiply(joint_coordinates[i][1],[640, 480]).astype(int)),
              cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA
          )
      ###########################
      #############################
    
    except Exception as e:
        print('error',e)
        pass

    # Flip the image horizontally for a selfie-view display.
    flipped_frame = cv2.flip(image, 0)

    cv2.imshow('MediaPipe Holistic',image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()